### Call individual functions for the component they are testing (currently does not need to do anything else)

In [ ]:
# Should read in files that would be used for testing (should represent idealized output that the function would receive)
# Results should be saved in a variable so they can be evaluated


### Testing Functions

In [ ]:
!pip install mne
!pip install umap-learn
!pip install pyedflib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.4-py3-none-any.whl size=86770 sha256=a423c9c53536f2612f9c6a4faab3c723bd72ed6f2b29557f0ccd0db78a4ec30d
  Stored in directory: /root/.cache/pip/wheels/fb/66/29/199acf5784d0f7b8add6d466175ab45506c96e386ed5dd0633
  Created wheel for pynndescent: filename=pynndescent-0.5.10-py3-none-any.whl size=55615 sha256=3ede85952a6fd2c07aab49b34d6f3340e6a8737d378b09bf9f5c8f9314573c7c
  Stored in directory: /root/.cache/pip/wheels/4a/38/5d/f60a40a66a9512b7e5e83517ebc2d1b42d857be97d135f1096
Successfully built umap-learn pynndescent
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.9 MB/s eta 0:00:00


In [ ]:
import mne
import pandas as pd
import numpy as np
import scipy as sp
import scipy.io as sio
from scipy.signal import periodogram
import pywt
from collections import Counter
from tqdm.notebook import tqdm
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from glob import glob
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import shutil
from scipy.signal import welch
#from utils import *
import logging
import zipfile
import pyedflib
from pyedflib import highlevel
import io
import tempfile

In [ ]:
#have to mount drive to access files in google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Import processing functions

# [UPDATE PATH]
%cd /content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/seizureai-main/

# Now, you can import your module
from dataset import *

/content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/seizureai-main
/Users/andresmichel/Documents/EGG data /v2.0.0/edf/dev


In [ ]:
# In the future this will be a full folder, but for now it will be one edf file

# [UPDATE PATH]
data_file_path  = '/content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/aaaaaajy_s001_t000.edf'
labels = [0]

In [ ]:
# Import functions

# [UPDATE PATH]
%cd  /content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/

from classical_ml_models import *
from get_features import *
from train_test_tune import *
from cnn import *
from rnn import *
from validate import *

/content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code


In [ ]:
# Import processing functions

# [UPDATE PATH]
%cd /content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/seizureai-main/

# Now, you can import your module
from dataset import *

/content/gdrive/Shareddrives/BE_223A_Seizure_Project/Code/seizureai-main


### Pre-Processing

In [ ]:
def eval_pre_processing(edf_path):

  eeg_data_pair = EEGDataPair(data_file_path)

  # Run the preprocessing pipeline
  edf_file = eeg_data_pair.processing_pipeline()

  # Store the original raw for visualization
  raw_before = eeg_data_pair.raw.copy()

  # Channels that are present after preprocessing
  common_chs = [ch for ch in raw_before.ch_names if ch in eeg_data_pair.raw.ch_names]

  # Same chanels for both plots
  raw_before.pick_channels(common_chs)
  eeg_data_pair.raw.pick_channels(common_chs)

  # Visualize EEG data BEFORE preprocessing
  raw_before.plot(title="Before Preprocessing", n_channels=20, scalings="auto", show=True)

  # Visualize EEG data AFTER preprocessing
  eeg_data_pair.raw.plot(title="After Preprocessing", n_channels=20, scalings="auto", show=True)

  print(edf_file.shape)

  print(edf_file.size)

  return edf_file

### Feature Extraction

In [1]:
# Test the get_features function by asserting it outputs a matrix with the right dimensions
def eval_get_features(list_signals, wavelet_name):
  features = get_features(list_signals, wavelet_name)
  print('The features matrix has dimensions ' + str(features.shape))
  if features.shape == (32, 177):
    print('Those are the correct dimensions!')
  if features.shape != (32, 177):
    print('Those are the wrong dimensions! The correct dimensions are (32, 177)')
  return features

### Classical ML Parameter Tuning

In [ ]:
def eval_tuning(data, labels):
  parameters = train_test_tune(data, labels)
  svm_param = parameters(1)
  rf_param = parameters(2)
  hmm_param = parameters(3)
  kmeans_param = parameters(4)
  umap_param = parameters(5)

  if len(svm_param) != (4):
    print('Incorrect SVM parameters')

  if len(rf_param) != (3):
    print('Incorrect RF parameters')

  if len(hmm_param) != (1):
    print('Incorrect HMM parameters')

  if len(kmeans_param) != (1):
    print('Incorrect K means paramters')

  if len(umap_param) != (4):
    print('Incorrect UMAP parameters')

  return parameters


### Deep Learning

In [ ]:
def eval_rnn(data, labels, val_data, parameters):
  predictions = rnn_model(data,labels, val_data,parameters)
  return predictions


In [ ]:
def eval_cnn(edf_file, labels):
  weights = run_cnn(edf_file, labels)
  return weights

### Validation

In [ ]:
def eval_validation(train_data, train_labels, val_data, val_labels, parameters):

  results = validate(train_data, train_labels, val_data, val_labels, parameters)

  # Check that validation methods are working and returning results in proper format

  return results